# TensorFlow: Tabular Classify Binary

*Detecting Naval Mines with Binary Classification of Sonar Data.*

![mines](../../../images/banner/mines.png)

In [1]:
cd '~/Desktop/AIQC'

/Users/layne/Desktop/AIQC


---

## Example Data

Reference [Example Datasets](example_datasets.ipynb) for more information.

This dataset is comprised of:

* *Features* = sonar readings that have been bounced off a distant object. 
* *Label* = either a rock or metal structure (potentially a naval mine).

In [2]:
from aiqc import datum
from aiqc.orm import Dataset

In [5]:
file_path = datum.get_path('sonar.csv')
shared_dataset = Dataset.Tabular.from_path(file_path=file_path, file_format='csv')

---

## Pipeline

Reference [High-Level API Docs](api_high_level.ipynb) for more information.

In [4]:
from aiqc.mlops import Pipeline, Input, Target, Stratifier
from sklearn.preprocessing import LabelBinarizer, PowerTransformer

In [6]:
pipeline = Pipeline(
    Input(
        dataset  = shared_dataset,
        encoders = Input.Encoder(
            PowerTransformer(method='yeo-johnson', copy=False),
            dtypes = ['float64']
        )
    ),
        
    Target(
        dataset = shared_dataset,
        column  = 'object',
        encoder = Target.Encoder(LabelBinarizer(sparse_output=False))
    ),

    Stratifier(
        size_test       = 0.12,
        size_validation = 0.22
    )
)

---

## Modeling

Reference [High-Level API Docs](api_high_level.ipynb) for more information.

In [7]:
from aiqc.mlops import Experiment, Architecture, Trainer
import tensorflow as tf
from tensorflow.keras import layers as l

In [8]:
def fn_build(features_shape, label_shape, **hp):
    m = tf.keras.models.Sequential()
    m.add(l.Input(shape=features_shape))
    m.add(l.Dense(hp['neuron_count'], activation='relu', kernel_initializer='he_uniform'))
    m.add(l.Dropout(0.40))
    m.add(l.Dense(hp['neuron_count'], activation='relu', kernel_initializer='he_uniform'))
    m.add(l.Dropout(0.40))
    m.add(l.Dense(units=label_shape[0], activation='sigmoid', kernel_initializer='glorot_uniform'))
    return m

In [9]:
def fn_train(
    model, loser, optimizer,
    train_features, train_label,
    eval_features, eval_label,
    **hp
):
    model.compile(
        loss        = loser
        , optimizer = optimizer
        , metrics   = ['accuracy']
    )
    model.fit(
        train_features, train_label
        , validation_data = (eval_features, eval_label)
        , verbose         = 0
        , batch_size      = 3
        , epochs          = hp['epochs']
        , callbacks       = [tf.keras.callbacks.History()]
    )
    return model

In [10]:
hyperparameters = dict(
    neuron_count=[24, 32], 
    epochs=[25, 50]
)

In [11]:
experiment = Experiment(
    Architecture(
        library         = "keras",
        analysis_type   = "classification_binary",
        fn_build        = fn_build,
        fn_train        = fn_train,
        hyperparameters = hyperparameters
    ),
    
    Trainer(
        pipeline     = pipeline,
        repeat_count = 2
    )
)

In [12]:
experiment.run_jobs()

🔮 Training Models 🔮: 100%|██████████████████████████████████████████| 8/8 [01:38<00:00, 12.29s/it]


---

## Visualization & Interpretation

For more information on visualization of performance metrics, reference the [Dashboard](dashboard.html) documentation.